In [53]:
import cv2
import numpy as np
import mediapipe as mp

In [54]:
image = cv2.imread("face_an_1.jpg")

In [55]:
def landmarksDetection(img, results, draw=False):
    img_height, img_width = img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [
        (int(point.x * img_width), int(point.y * img_height))
        for point in results.multi_face_landmarks[0].landmark
    ]
    if draw:
        [cv2.circle(img, p, 2, (0, 255, 0), -1) for p in mesh_coord]

    # returning the list of tuples for each landmarks
    return mesh_coord

In [56]:
map_face_mesh = mp.solutions.face_mesh
with map_face_mesh.FaceMesh(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as face_mesh:
    frame = cv2.resize(image, None, fx=1, fy=1, interpolation=cv2.INTER_CUBIC)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    results = face_mesh.process(rgb_frame)
    mesh_coords = landmarksDetection(frame, results, False)

In [2]:
LEFT_EYE = [
    362,
    382,
    381,
    380,
    374,
    373,
    390,
    249,
    263,
    466,
    388,
    387,
    386,
    385,
    384,
    398,
]
RIGHT_EYE = [
    33,
    7,
    163,
    144,
    145,
    153,
    154,
    155,
    133,
    173,
    157,
    158,
    159,
    160,
    161,
    246,
]
FACE_OVAL = [
    10,
    338,
    297,
    332,
    284,
    251,
    389,
    356,
    454,
    323,
    361,
    288,
    397,
    365,
    379,
    378,
    400,
    377,
    152,
    148,
    176,
    149,
    150,
    136,
    172,
    58,
    132,
    93,
    234,
    127,
    162,
    21,
    54,
    103,
    67,
    109,
]
EYE = [
    362,
    382,
    381,
    380,
    374,
    373,
    390,
    249,
    263,
    466,
    388,
    387,
    386,
    385,
    384,
    398,
    33,
    7,
    163,
    144,
    145,
    153,
    154,
    155,
    133,
    173,
    157,
    158,
    159,
    160,
    161,
    246,
    ]

top_head = mesh_coords[10]
bot_chin = mesh_coords[152]
face_coords = [mesh_coords[p] for p in FACE_OVAL]
left_coords = [mesh_coords[p] for p in LEFT_EYE]
right_coords = [mesh_coords[p] for p in RIGHT_EYE]
eye_coords = [mesh_coords[p] for p in EYE]

In [58]:
# Tính trung tâm của hai mắt
eye_center = (round(sum(coord[0] for coord in eye_coords) / len(eye_coords)), round(sum(coord[1] for coord in eye_coords) / len(eye_coords)))
face_center = (round(sum(coord[0] for coord in face_coords) / len(face_coords)), round(sum(coord[1] for coord in face_coords) / len(face_coords)))

print(eye_center)
print(face_center)

(231, 157)
(241, 170)


In [59]:
slope_rotate_head = (top_head[1] - bot_chin[1]) / (top_head[0] - bot_chin[0])
slope_direct = (eye_center[1] - face_center[1]) / (eye_center[0] - face_center[0])

In [60]:
a = np.arctan(slope_rotate_head) * 180 / np.pi
if a > 0:
    angle = -(90 - abs(a))
else: 
    angle = 90 - a

In [61]:
w_new = round(abs(frame.shape[1] * np.cos(np.deg2rad(angle))) + abs(frame.shape[0] * np.sin(np.deg2rad(angle))))
h_new = round(abs(frame.shape[1] * np.sin(np.deg2rad(angle))) + abs(frame.shape[0] * np.cos(np.deg2rad(angle))))

In [62]:
frame.shape

(300, 500, 3)

In [63]:
def translate_image_to_center(image, new_size):
    curr_size = (image.shape[1], image.shape[0])
    new_image = np.zeros((new_size[1], new_size[0]) + (3,), dtype=np.uint8)

    new_image[(new_size[1] - curr_size[1])//2:(new_size[1] + curr_size[1])//2,
               (new_size[0] - curr_size[0])//2:(new_size[0] + curr_size[0])//2] = image
    return new_image

In [64]:
new_size = (w_new, h_new)
def rotate_image(image, new_sizem, angle):
    frame = translate_image_to_center(image, new_size)
    new_center = (w_new // 2, h_new // 2)
    rotation_matrix = cv2.getRotationMatrix2D(new_center, angle, 1.0)
    frame = cv2.warpAffine(frame, rotation_matrix, new_size)
    return frame

In [65]:
def rotate_point(point, angle, center, new_size, old_size):
    new_point = (point[0] + (new_size[0] - old_size[0]) // 2, point[1] + (new_size[1] - old_size[1]) // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotation_matrix = np.vstack([rotation_matrix, [0, 0, 1]])
    point_homogeneous = np.array([new_point[0], new_point[1], 1]).reshape((3, 1))
    rotated_point = np.dot(rotation_matrix, point_homogeneous)
    return round(rotated_point[0][0]), round(rotated_point[1][0])

In [67]:
new_chin = rotate_point(bot_chin, angle, (w_new // 2, h_new // 2), new_size, (image.shape[1], image.shape[0]))
new_chin[0]

270

In [68]:
frame = rotate_image(image, new_size, angle)
cv2.circle(frame, new_chin, 5, (0, 255, 0), -1)

cv2.imshow("Rotated Image", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()